In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt

import sys

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
import statsmodels.api as sm

import datetime
from dateutil.parser import *

from sklearn.pipeline import Pipeline
from svi_transformeri import dayName, dummy_days, DFStandardScaler, ZeroFillTransformer, DFFeatureUnion

import pickle

In [2]:
categorical_features = ['day_of_week']
numerical_features = ['regular_price', 'discounted_price','number_of_stores']

In [3]:
def removing_outliers(self):
    upper_lim = self['amount'].quantile(.95)
    lower_lim = self['amount'].quantile(.05)
    self=self[(self['amount'] < upper_lim) & (self['amount'] > lower_lim)]
    return self

### 0.1.4. Separating training and test data

In [10]:
def X_train_test_separation(self):
    aux = self.loc[self.index.get_level_values('date') <= '2018-01-01']
    train = aux.drop('amount',axis=1)
    
    aux = self.loc[self.index.get_level_values('date') > '2018-01-01']
    test = aux.drop('amount',axis=1)
    
    return train, test

In [11]:
def y_train_test_separation(self):
    aux = self.loc[self.index.get_level_values('date') <= '2018-01-01']
    train = aux['amount']
    
    aux = self.loc[self.index.get_level_values('date') > '2018-01-01']
    test = aux['amount']
    
    return train, test

In [4]:
pipeline = Pipeline([ ('features', DFFeatureUnion([       
                                                      ('categoricals', Pipeline([ ('dani',dayName()), ('dummyD', dummy_days())]))
                                                  ])), 
                      ('scaler', DFStandardScaler()) 
                   ])

TypeError: __init__() takes 1 positional argument but 2 were given

In [9]:
saving pipeline to picikle

pipeline_pickle_path = 'pipeline_pickle.pkl'
pipeline_pickle = open (pipeline_pickle_path,'wb')
pickle.dump(pipeline, pipeline_pickle)
pipeline_pickle.close()

Loading pipeline from pipeline_pickle:

In [10]:
pipeline_pickle_path = 'pipeline_pickle.pkl'
pipeline_unpickle = open(pipeline_pickle_path, 'rb')
  
pipeline_from_pickle = pickle.load(pipeline_unpickle)

In [11]:
pipeline_from_pickle

Pipeline(memory=None,
         steps=[('features',
                 <transformers.DFFeatureUnion object at 0x000002AFD112CCC0>),
                ('scaler',
                 <transformers.DFStandardScaler object at 0x000002AFD112CEF0>)],
         verbose=False)

## 0.3. Data loading

In [12]:
X = pd.read_parquet("C:/Users/vvrhovec/Veronika/kodovi/radni_folder/parquet/X_prices_stores_amount.parquet")

C:\Users\vvrhovec\AppData\Local\Continuum\miniconda3\envs\okolina\lib\site-packages\pyarrow\pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\vvrhovec\AppData\Local\Continuum\miniconda3\envs\okolina\lib\site-packages\pyarrow\pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\vvrhovec\AppData\Local\Continuum\miniconda3\envs\okolina\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


Napomena: srediti parquet fajlove za učitavanje svakog od artikla

In [13]:
X_38 = X.loc[38]
X_39 = X.loc[39]
X_40 = X.loc[40]
X_41 = X.loc[41]
X_57 = X.loc[57]

**<hr>**

# <center>**ITEMS**</center>

**<hr>**

# 1. ITEM 38

In [18]:
X_38.describe()

,regular_price,discounted_price,number_of_stores,amount
count,733.000000,733.000000,733.000000,733.000000
mean,0.099944,0.099944,420.538881,245.199181
std,0.000247,0.000247,21.571971,125.344983
min,0.098000,0.098000,368.000000,43.000000
25%,0.100000,0.100000,402.000000,143.000000
50%,0.100000,0.100000,420.000000,229.000000
75%,0.100000,0.100000,442.000000,339.000000
max,0.100000,0.100000,452.000000,552.000000


## 1.1. Data preprocessing

### 1.1.1. Removing outliers

In [17]:
X_38 = removing_outliers(X_38)

### 1.1.2. Separating data into training and test set

In [21]:
X_38_train, X_38_test =  X_train_test_separation(X_38)
y_38_train, y_38_test =  y_train_test_separation(X_38)

In [186]:
# X_38_train.to_parquet('X_38_train.parquet')
# X_38_test.to_parquet('X_38_test.parquet')

### 1.1.3. Pipeline

In [22]:
X_38_train = pipeline_from_pickle.fit_transform(X_38_train)
X_38_test = pipeline_from_pickle.transform(X_38_test)

In [23]:
print (X_38_train.shape, y_38_train.shape)
print (X_38_test.shape, y_38_test.shape)

(620, 10) (620,)
(113, 10) (113,)
